# Zooniverse: View a TESS Light Curve Using a TIC ID

This Notebook is primarily aimed at citizen scientists examining TESS light curves for evidence of transiting exoplanets. To that end, this is a "single cell" Notebook; you can modify the target and run the cell to produce a graph. This is done for convenience, but we also include a step-by-step walkthrough for those curious about what is "happening behind the scenes".

## Table of Contents

* [One-step TESS Light Curve](#One-step-TESS-Light-Curve)
* [Detailed Light Curve Walkthrough](#Detailed-Light-Curve-Walkthrough)
* [Further Reading](#Further-Reading)


## One-step TESS Light Curve
To create a light curve for your target, edit the fields below. Replace the number in quotes after `tic_id =` with your desired target. Also make sure to update the `sector =` to an appropriate value.

Do NOT edit any of the other lines. Again, if you're interested in learning about how this code works, see the rest of the Notebook below.

Select the cell to edit, then press `shift + enter` to run it.

In [ ]:
# EDIT THESE FIRST TWO LINES
tic_id = "81609273"
sector = 62

########################################
# Do NOT edit any of the following lines
########################################
from astropy.io import fits
from astroquery.mast import Observations

import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 600
Observations.enable_cloud_dataset()

obs = Observations.query_criteria(target_name=tic_id, obs_collection="TESS", sequence_number=sector)

if len(obs)==0:
    raise UserWarning("No matching observations. Are you certain the target was observed in this sector?")

files = Observations.get_product_list(obs)
fits_file = Observations.download_products(files, description="Light curves")

with fits.open(fits_file['Local Path'][0], mode="readonly") as hdulist:
    tess_bjds = hdulist[1].data['TIME']
    sap_fluxes = hdulist[1].data['SAP_FLUX']
    pdcsap_fluxes = hdulist[1].data['PDCSAP_FLUX']
    
fig, ax = plt.subplots(figsize=(10, 4))
ax.scatter(tess_bjds, pdcsap_fluxes, s=1, color='r')
fig.suptitle(f"TIC {tic_id}, Sector {sector}, PDCSAP Flux")
ax.set_ylabel("PDCSAP Flux (e-/s)")
ax.set_xlabel("Time (TBJD)")
plt.show()

fig, ax = plt.subplots(figsize=(10,4))
ax.scatter(tess_bjds, sap_fluxes, s=1, color='k')
fig.suptitle(f"TIC {tic_id}, Sector {sector}, SAP Flux")
ax.set_ylabel("SAP Flux (e-/s)")
ax.set_xlabel("Time (TBJD)")
plt.show()

Potential issues and how to solve them:
* Did you get a 'No matching observations' error in the above cell? Make sure that your target was observed in the sector you specified.
* Cell is running, but seems "stuck"? Press the square in the menu bar, to the right of the triangle "play" icon. This will interrupt the cell you can try to run it again.


## Detailed Light Curve Walkthrough

Thanks for being curious! Let's take a look at what exactly is happening in that tangle of code above, step-by-step. For the uninitiated, we're using the [Python programming lanuage](https://www.python.org/about/gettingstarted/).

### Target Information

To start, we save the target TIC ID and sector. We'll use this information later to perform a search, but it's nice to have it at the top for easy editing. Notice that we save `tic_id` as a string (text value, indicated by quotes) whereas sector is an integer. The way the rest of the code is written, the type (string or integer) actually makes no difference to the output.

In [ ]:
tic_id = "81609273"
sector = 62

### Imports and Adjustments

Generally, when writing code the import statements are the very first things written. An "import" adds code not included in default (or "base") Python. For example, [astroquery.mast](https://astroquery.readthedocs.io/en/latest/mast/mast.html) is a package that makes it easy to search for astronomy data in the [MAST Archive](https://archive.stsci.edu). Since most people are not astronomers, this is not a part of base Python. Let's be explicit about why we need these imports:
* `astropy.io` : we need this to open the FITS files containing our TESS data
* `astroquery.mast` : this makes it easy to query MAST for TESS data
* `matplotlib` : we only do this import so that we can change the DPI below
* `matplotlib.pyplot` : this gives us the tools for making pleasant-looking plots

Below the imports, we make some minor adjustments to the DPI, to increase the resolution of our graphs, and "enable cloud datasets". This second piece is important: TIKE (the platform you're currently on) is hosted in the cloud. Reading data from our cloud archive is therefore much faster than trying to read from our servers in Baltimore.

In [ ]:
from astropy.io import fits
from astroquery.mast import Observations

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.dpi'] = 600
Observations.enable_cloud_dataset()

### Searching for Observations
This next line is the most important in the entire Notebook. `Observations.query_criteria` is a powerful way to search MAST, using [Observation metadata](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html). There are many criteria you can search on, so let's focus on the three we use here.

1. `target_name` is not often used, as you would need to know in advance _exactly_ how the observer or mission labeled it. Fortunately in this case, the TESS mission uses a standard set of names from the [TESS Input Catalog](https://tess.mit.edu/science/tess-input-catalogue/). Be careful using this field for other searches!
2. `obs_collection` translates to "mission". Specifying TESS greatly reduces the number of Observations to search through which in turns makes our search faster.
3. `sequence_number` is used for Kepler/K2/TESS quarters and sectors.

Note that we are using the variables from above here, but you could use the actual values. For example, you could specify `sequence_number=61`.

In [ ]:
obs = Observations.query_criteria(target_name=tic_id, obs_collection="TESS", sequence_number=sector)
obs

You should see a list of all matching Observations in a table above.

### Getting Products

Once we have an Observation, we need to get the associated products: the actual files containing our data. Let's request all of the files and see what is returned.

In [ ]:
files = Observations.get_product_list(obs)
files

Woah! We had one Observations, but that's turned into 8 files. We really just want the light curves, but it's important to know that you can access the raw images (target pixel files) and data summaries. In fact, TESS [produces a wide variety of data products](https://outerspace.stsci.edu/display/TESS/2.0+-+Data+Product+Overview).

For now, let's just load the light curves.

In [ ]:
fits_file = Observations.download_products(files, description="Light curves")

### Reading a FITS File
The structure of FITS files can be unintuitive at first; but that's to be expected from a 40 year old file format! Let's take a look at a little bit of that now.

In [ ]:
file_path = fits_file['Local Path'][0]
fits.info(file_path)

We're interested in the `LIGHTCURVE` data, which is on HDU (Header Data Unit) 1. HDU1 is broken up further, but it can be a bit confusing to look at. If you want, you can read all of the available columns in No. 1 by running the cell below; be forewarned that it's a lot of text!

In [ ]:
# Warning! This prints out a lot of text
with fits.open(file_path, mode="readonly") as hdulist:
    header1 = hdulist[1].header
    print(repr(header1))

Now let's get our data out of the file. We're interested in the flux (brightness) of a star over time; we need to get the timestamps and the associated fluxes to create our plot.

**Note: We grab the SAP and PDCSAP fluxes, for convenience.** PDCSAP is generally better for planet hunting, but there are valid reasons to use SAP as well!

In [ ]:
with fits.open(file_path, mode="readonly") as hdulist:
    tess_bjds = hdulist[1].data['TIME']
    sap_fluxes = hdulist[1].data['SAP_FLUX']
    pdcsap_fluxes = hdulist[1].data['PDCSAP_FLUX']

### Making a Plot
Finally, we can create our figures. We're going to plot the PDCSAP and SAP fluxes separately.

In [ ]:
# Create figure and axis
fig, ax = plt.subplots(figsize=(10,4))

# Plot the timeseries in black circles
ax.scatter(tess_bjds, pdcsap_fluxes, s=1, color='k')

# Let's label the axes and define a title for the figure
fig.suptitle(f"TIC {tic_id}, Sector {sector}, PDCSAP Flux")
ax.set_ylabel("PDCSAP Flux (e-/s)")
ax.set_xlabel("Time (TBJD)")

# Show the figure
plt.show()

In [ ]:
# Create figure and axis
fig, ax = plt.subplots(figsize=(10,4))

# Plot the timeseries
ax.scatter(tess_bjds, sap_fluxes, s=1, color='k')

# Add labels, title
fig.suptitle(f"TIC{tic_id}, Sector {sector}, PDCSAP Flux")
ax.set_ylabel("SAP Flux (e-/s)")
ax.set_xlabel("Time (TBJD)")

# Show the figure
plt.show()

And done. We've successfully plotted a TESS light curve from data at MAST. Looks like we've found an eclipsing binary.

Happy planet hunting!

## Further Reading
Still curious about TESS and lightcurves? Check out our tutorials in the [MAST Notebook repository](https://spacetelescope.github.io/mast_notebooks/intro.html).

## About this Notebook
For support, please contact the Archive HelpDesk at archive@stsci.edu, or through the [MAST HelpDesk Portal](https://stsci.service-now.com/mast).

**Author:** Thomas Dutkiewicz <br>
**Keywords:** TIKE, AWS Cloud, Light curves <br>
**Last Updated:** May 2023 <br>
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 